In [ ]:
import numpy as np
import rasterio
import math
import os
import logging
from tqdm import tqdm
import fiona
from fiona.crs import CRS
from shapely.geometry import Point, LineString, mapping, shape
from shapely.ops import unary_union
from numba import njit
from datetime import datetime
from skimage.draw import disk
from rasterio.enums import Resampling
from rasterio.features import rasterize

# --- 1. CONFIGURACIÓN DEL PROYECTO Y PARÁMETROS ---
# Carpeta principal donde se almacenarán todas las sesiones y el log principal.
BASE_PROCESSING_FOLDER = R"C:\Users\User\Desktop\SofiHanna\PROCESAMIENTO_LCP"

# --- Rutas a los datos de entrada ---
POINTS_SUBFOLDER = os.path.join(R"C:\Users\User\Desktop\SofiHanna", 'puntos_por_100km')
COST_RASTER_PATH = os.path.join(R"C:\Users\User\Desktop\SofiHanna", 'final_total_cost.tif')
ALL_POINTS_SHAPEFILE = os.path.join(POINTS_SUBFOLDER, 'puntos_por_100km.shp')
MASK_SHAPEFILE_PATH = os.path.join(R"C:\Users\User\Desktop\SofiHanna", 'poligono_extendido_cortado_disuelto.shp')

# --- PARÁMETROS DE CÁLCULO ---
ORIGIN_POINT_ID = 5
ID_FIELD_NAME = 'rand_point'
DOWNSAMPLING_FACTORS = [32, 20, 10]
CORRIDOR_BUFFER_PIXELS = 150
HEURISTIC_WEIGHT = 1.0

# --- CLASE AUXILIAR PARA INTEGRAR LOGGING CON TQDM ---
class TqdmLoggingHandler(logging.Handler):
    def __init__(self, level=logging.NOTSET):
        super().__init__(level)
    def emit(self, record):
        try:
            msg = self.format(record)
            tqdm.write(msg)
            self.flush()
        except (KeyboardInterrupt, SystemExit): raise
        except Exception: self.handleError(record)

# --- 2. FUNCIONES AUXILIARES ---
def load_all_points(shapefile_path, id_field):
    points = {}; crs = None
    try:
        with fiona.open(shapefile_path, 'r') as c:
            crs = c.crs
            if len(c) == 0: logging.error(f"'{os.path.basename(shapefile_path)}' está vacío."); return None, None
            for f in c: points[int(f['properties'][id_field])] = f['geometry']['coordinates']
        return points, crs
    except Exception as e: logging.error(f"Error cargando puntos: {e}"); return None, None

def load_polygon_geometry(shapefile_path):
    if not shapefile_path or not os.path.exists(shapefile_path): return None
    try:
        with fiona.open(shapefile_path, 'r') as c:
            if len(c) == 0: logging.warning(f"'{os.path.basename(shapefile_path)}' de máscara está vacío."); return None
            return unary_union([shape(f['geometry']) for f in c])
    except Exception as e: logging.error(f"Error cargando polígono: {e}"); return None

def world_to_pixel(transform, x, y):
    col, row = ~transform * (x, y); return int(row), int(col)

def create_mask_from_vector(vector_path, raster_src):
    if not vector_path or not os.path.exists(vector_path):
        logging.warning("No se encontró archivo de máscara."); return None
    with fiona.open(vector_path, "r") as vf:
        if vf.crs != raster_src.crs:
            logging.critical(f"CRS de máscara ({vf.crs}) y ráster ({raster_src.crs}) no coinciden."); return None
        shapes = [f["geometry"] for f in vf]
    return rasterize(shapes, out_shape=raster_src.shape, transform=raster_src.transform, fill=0, all_touched=True, dtype=np.uint8).astype(bool)

# --- 3. ALGORITMO A* Y HELPERS ---
@njit
def heuristic_numba(r1, c1, r2, c2, dx, dy): return math.sqrt((r2 - r1)**2 + (c2 - c1)**2) * math.sqrt(dx*dy)
@njit
def a_star_numba_compiled(cost_array, nodata_value, start_pixel, end_pixel, dx, dy, weight, search_mask):
    height, width = cost_array.shape; g_cost = np.full(cost_array.shape, np.inf); came_from = np.full(cost_array.shape, -1, dtype=np.int16)
    open_set = np.zeros((1, 4)); h_initial = heuristic_numba(start_pixel[0], start_pixel[1], end_pixel[0], end_pixel[1], dx, dy)
    open_set[0] = [h_initial*weight, 0.0, start_pixel[0], start_pixel[1]]; g_cost[start_pixel] = 0; path_found = False
    while open_set.shape[0] > 0:
        min_idx = np.argmin(open_set[:, 0]); f, g, r, c = open_set[min_idx]; current_pos = (int(r), int(c))
        if open_set.shape[0] == 1: open_set = np.zeros((0, 4))
        else: open_set = np.vstack((open_set[:min_idx], open_set[min_idx + 1:]))
        if current_pos == end_pixel: path_found = True; break
        if g > g_cost[current_pos]: continue
        cost_current = cost_array[current_pos]
        for dr in range(-1, 2):
            for dc in range(-1, 2):
                if dr == 0 and dc == 0: continue
                neighbor_pos = (current_pos[0] + dr, current_pos[1] + dc)
                if not (0 <= neighbor_pos[0] < height and 0 <= neighbor_pos[1] < width): continue
                if not search_mask[neighbor_pos]: continue
                cost_neighbor = cost_array[neighbor_pos];
                if cost_neighbor == nodata_value: continue
                dist_m = math.sqrt((dr*dy)**2 + (dc*dx)**2); avg_cost = (cost_current + cost_neighbor) / 2.0
                tentative_g_cost = g + (avg_cost*dist_m)
                if tentative_g_cost < g_cost[neighbor_pos]:
                    direction = (dr+1)*3 + (dc+1); came_from[neighbor_pos] = direction; g_cost[neighbor_pos] = tentative_g_cost
                    h = heuristic_numba(neighbor_pos[0], neighbor_pos[1], end_pixel[0], end_pixel[1], dx, dy)
                    new_f_cost = tentative_g_cost + (h*weight)
                    new_entry = np.array([[new_f_cost, tentative_g_cost, neighbor_pos[0], neighbor_pos[1]]])
                    open_set = np.vstack((open_set, new_entry))
    return path_found, came_from, g_cost
@njit
def reconstruct_path_pixels_numba(came_from_array, start_pixel, end_pixel):
    path = np.zeros((came_from_array.size, 2), dtype=np.int32); current_pos_r, current_pos_c = end_pixel; count = 0; limit = came_from_array.size
    while (current_pos_r, current_pos_c) != start_pixel and count < limit:
        path[count] = np.array([current_pos_r, current_pos_c]); direction = came_from_array[current_pos_r, current_pos_c]
        if direction == -1: return None
        dc = (direction % 3) - 1; dr = (direction // 3) - 1; current_pos_r -= dr; current_pos_c -= dc; count += 1
    path[count] = np.array([start_pixel[0], start_pixel[1]]); return path[:count+1][::-1]

# --- 4. FUNCIONES PARA BÚSQUEDA JERÁRQUICA ---
def create_low_res_data(src, factor):
    low_res_shape = (src.height // factor, src.width // factor)
    low_res_data = src.read(1, out_shape=low_res_shape, resampling=Resampling.average)
    low_res_transform = src.transform * src.transform.scale(factor, factor)
    return low_res_data, low_res_transform, src.res[0]*factor, src.res[1]*factor
def create_search_corridor(path_low_res, high_res_shape, factor, buffer_pixels):
    corridor_mask = np.zeros(high_res_shape, dtype=bool);
    if path_low_res is None: return corridor_mask
    for r_low, c_low in path_low_res:
        r_high = int(r_low*factor + factor/2); c_high = int(c_low*factor + factor/2)
        rr, cc = disk((r_high, c_high), buffer_pixels, shape=high_res_shape); corridor_mask[rr, cc] = True
    return corridor_mask
def save_path_to_shapefile(pixel_path, transform, crs, output_path):
    if pixel_path is None or not pixel_path.any():
        logging.warning(f"No se guardará {os.path.basename(output_path)}, ruta vacía."); return
    world_coords = [transform * (p[1] + 0.5, p[0] + 0.5) for p in pixel_path]
    schema = {'geometry': 'LineString', 'properties': {'id': 'str'}}
    with fiona.open(output_path, 'w', 'ESRI Shapefile', schema, crs=CRS.from_wkt(crs.to_wkt()) if crs else None) as c:
        c.write({'geometry': mapping(LineString(world_coords)), 'properties': {'id': os.path.basename(output_path)}})

# --- 5. EJECUCIÓN DEL ANÁLISIS ---
if __name__ == '__main__':
    os.makedirs(BASE_PROCESSING_FOLDER, exist_ok=True)
    
    # --- Configurar el logger maestro persistente ---
    log_file_path = os.path.join(BASE_PROCESSING_FOLDER, "registro_maestro_procesamiento.log")
    log = logging.getLogger(); log.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    if not log.handlers:
        file_handler = logging.FileHandler(log_file_path, mode='a'); file_handler.setFormatter(formatter); log.addHandler(file_handler)
        tqdm_handler = TqdmLoggingHandler(); tqdm_handler.setFormatter(formatter); log.addHandler(tqdm_handler)

    log.info("=========================================================")
    log.info("============== INICIANDO NUEVA EJECUCIÓN ==============")
    log.info("=========================================================")

    try:
        all_points, points_crs = load_all_points(ALL_POINTS_SHAPEFILE, ID_FIELD_NAME)
        if not all_points:
             raise ValueError("No se pudieron cargar los puntos de entrada. Verifique el log de errores.")

        # --- LÓGICA DE REANUDACIÓN AUTOMÁTICA ---
        num_expected_routes = len(all_points) - 1
        session_to_resume = None
        
        try:
            existing_sessions = [d for d in os.listdir(BASE_PROCESSING_FOLDER) if os.path.isdir(os.path.join(BASE_PROCESSING_FOLDER, d)) and d[:8].isdigit()]
        except FileNotFoundError:
            existing_sessions = []

        if existing_sessions:
            existing_sessions.sort()
            latest_session_name = existing_sessions[-1]
            latest_session_path = os.path.join(BASE_PROCESSING_FOLDER, latest_session_name)
            
            completed_routes = [f for f in os.listdir(latest_session_path) if f.startswith(f"ruta_final_desde_{ORIGIN_POINT_ID}_a_") and f.endswith(".shp")]
            
            if len(completed_routes) < num_expected_routes:
                session_to_resume = latest_session_name

        if session_to_resume:
            SESSION_OUTPUT_DIR = os.path.join(BASE_PROCESSING_FOLDER, session_to_resume)
            log.info(f"MODO REANUDACIÓN AUTOMÁTICA: La última sesión '{session_to_resume}' está incompleta. Reanudando...")
        else:
            if existing_sessions:
                log.info(f"La última sesión '{existing_sessions[-1]}' se encontró completa.")
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            session_folder_name = f"{timestamp}_desde_punto_{ORIGIN_POINT_ID}"
            SESSION_OUTPUT_DIR = os.path.join(BASE_PROCESSING_FOLDER, session_folder_name)
            log.info(f"MODO NUEVO: Creando nueva carpeta de sesión: {session_folder_name}")
        os.makedirs(SESSION_OUTPUT_DIR, exist_ok=True)
        # --- FIN DE LA LÓGICA DE REANUDACIÓN ---

        with rasterio.open(COST_RASTER_PATH) as src:
            log.info("Cargando superficie de coste de alta resolución en RAM...")
            cost_data_high_res = src.read(1)
            mask_polygon_geom = load_polygon_geometry(MASK_SHAPEFILE_PATH)
            if mask_polygon_geom:
                log.info("Creando máscara de búsqueda rasterizada...")
                search_mask_hr_user = create_mask_from_vector(MASK_SHAPEFILE_PATH, src)
                log.info("\n--- VERIFICACIÓN PRELIMINAR DE PUNTOS CONTRA LA MÁSCARA ---")
                invalid_points_ids = [str(pid) for pid, coords in all_points.items() if not mask_polygon_geom.contains(Point(coords))]
                if invalid_points_ids:
                    log.critical("¡ANÁLISIS CANCELADO! Puntos fuera de la máscara: " + ", ".join(invalid_points_ids)); exit()
                else: log.info("VERIFICACIÓN SUPERADA: Todos los puntos están dentro del área.")

            origin_coords = all_points.get(ORIGIN_POINT_ID)
            start_pixel_hr = world_to_pixel(src.transform, origin_coords[0], origin_coords[1])
            log.info(f"\nInicio del cálculo. Origen Fijo: Punto ID {ORIGIN_POINT_ID} -> Píxel {start_pixel_hr}")
            
            for dest_id, dest_coords in tqdm(all_points.items(), desc="Calculando rutas"):
                if dest_id == ORIGIN_POINT_ID: continue
                
                final_output_path = os.path.join(SESSION_OUTPUT_DIR, f"ruta_final_desde_{ORIGIN_POINT_ID}_a_{dest_id}.shp")
                if os.path.exists(final_output_path):
                    log.info(f"Ruta para {ORIGIN_POINT_ID}->{dest_id} ya existe. Omitiendo."); continue
                
                log.info(f"\n--- Calculando ruta desde {ORIGIN_POINT_ID} hasta {dest_id} ---")
                end_pixel_hr = world_to_pixel(src.transform, dest_coords[0], dest_coords[1])
                
                path_found_lr, path_pixels_lr, successful_factor, trans_low = False, None, None, None
                log.info("-> FASE 1: Búsqueda en baja resolución...")
                for factor in DOWNSAMPLING_FACTORS:
                    log.info(f"Probando con factor: {factor}...")
                    cost_data_lr, trans_lr, dx_lr, dy_lr = create_low_res_data(src, factor)
                    search_mask_lr = search_mask_hr_user[::factor, ::factor] if search_mask_hr_user is not None else np.ones(cost_data_lr.shape, dtype=bool)
                    start_lr = (start_pixel_hr[0]//factor, start_pixel_hr[1]//factor); end_lr = (end_pixel_hr[0]//factor, end_pixel_hr[1]//factor)
                    found, came_from_lr, _ = a_star_numba_compiled(cost_data_lr, src.nodata, start_lr, end_lr, dx_lr, abs(dy_lr), HEURISTIC_WEIGHT, search_mask_lr)
                    if found:
                        log.info(f"ÉXITO con factor {factor}."); path_found_lr, successful_factor, trans_low = True, factor, trans_lr
                        path_pixels_lr = reconstruct_path_pixels_numba(came_from_lr, start_lr, end_lr); break
                    else: log.info(f"FALLÓ con factor {factor}.")
                
                if not path_found_lr: log.warning(f"Fase 1 fallida para {ORIGIN_POINT_ID}->{dest_id}.")

                log.info("-> FASE 2: Búsqueda en alta resolución...")
                path_found_hr, came_from_hr = False, None
                if path_found_lr:
                    log.info("Creando corredor y buscando dentro...")
                    corridor_mask = create_search_corridor(path_pixels_lr, cost_data_high_res.shape, successful_factor, CORRIDOR_BUFFER_PIXELS)
                    final_mask = np.logical_and(corridor_mask, search_mask_hr_user) if search_mask_hr_user is not None else corridor_mask
                    path_found_hr, came_from_hr, _ = a_star_numba_compiled(cost_data_high_res, src.nodata, start_pixel_hr, end_pixel_hr, src.res[0], abs(src.res[1]), HEURISTIC_WEIGHT, final_mask)
                
                if not path_found_hr:
                    if path_found_lr: log.warning("Búsqueda en corredor falló.")
                    log.info("PLAN B: Realizando LCP sobre toda la máscara...")
                    fallback_mask = search_mask_hr_user if search_mask_hr_user is not None else np.ones_like(cost_data_high_res, dtype=bool)
                    path_found_hr, came_from_hr, _ = a_star_numba_compiled(cost_data_high_res, src.nodata, start_pixel_hr, end_pixel_hr, src.res[0], abs(src.res[1]), HEURISTIC_WEIGHT, fallback_mask)
                
                if path_found_hr:
                    log.info(f"-> ÉXITO: Ruta final encontrada para {ORIGIN_POINT_ID}->{dest_id}.")
                    path_pixels_hr = reconstruct_path_pixels_numba(came_from_hr, start_pixel_hr, end_pixel_hr)
                    if path_found_lr:
                        p1_path = os.path.join(SESSION_OUTPUT_DIR, f"ruta_fase1_desde_{ORIGIN_POINT_ID}_a_{dest_id}.shp")
                        save_path_to_shapefile(path_pixels_lr, trans_low, src.crs, p1_path)
                    save_path_to_shapefile(path_pixels_hr, src.transform, src.crs, final_output_path)
                else:
                    log.error(f"-> ERROR CRÍTICO: No se encontró ruta de alto nivel para {ORIGIN_POINT_ID}->{dest_id}.")
            
            log.info("\n¡Proceso de Uno a Todos completado!")

    except Exception as e:
        logging.critical(f"Ocurrió un error inesperado en el proceso principal: {e}", exc_info=True)
    
    log.info("================== EJECUCIÓN FINALIZADA ==================\n\n")